In [42]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

from tqdm import tqdm_notebook
from scipy.stats import linregress
from scipy.optimize import minimize

In [2]:
candidates = ['ЗеленськийВолодимир','ПорошенкоПетро']

dtype_dic = {'district_number': str}
df_results = pd.read_csv('00_data/elections_round2_v3.csv',dtype=dtype_dic)
df_round1 = pd.read_csv('00_data/elections_round1_v1.csv',dtype = dtype_dic)
df_results = df_results.rename(columns={'ЗеленськийВолодимир':'ЗеленськийВолодимир_round2',
                                        'ПорошенкоПетро':'ПорошенкоПетро_round2',
                                        '2.К-сть виборців, внесених до списку_y':'2.К-сть виборців, внесених до списку'})

df_results['participation_round2'] = (df_results[candidates[0]+'_round2'] + df_results[candidates[1]+'_round2'])/df_results['2.К-сть виборців, внесених до списку']

'''
On several smaller voting districts (e.g., hospitals) inflow of new voters registrations
led to paricipation > 1 (because we take registration at round 1 as a baseline).
Therefore, is such cases minor adjustment is needed
'''
def adjust_participation(x):
    if x['participation_round2']>1:
        x['ЗеленськийВолодимир_round2'] = x['ЗеленськийВолодимир_round2']/x['participation_round2']
        x['ПорошенкоПетро_round2'] = x['ПорошенкоПетро_round2']/x['participation_round2']
        x['participation_round2'] = (x['ЗеленськийВолодимир_round2']+x['ПорошенкоПетро_round2'])/x['2.К-сть виборців, внесених до списку']
    return x['ЗеленськийВолодимир_round2'],x['ПорошенкоПетро_round2'],x['participation_round2']

df_results['ЗеленськийВолодимир_round2'], df_results['ПорошенкоПетро_round2'], df_results['participation_round2'] = zip(*df_results.apply(lambda x:adjust_participation(x),axis=1))
df_results.head()

print(df_results.shape)
print(df_round1.shape)
df_results.head()

(29608, 21)
(29989, 54)


,district_number,district_gen_numeric,district_general,district_location,1.К-сть одержаних бюлетенів,"2.К-сть виборців, внесених до списку_x","3.К-сть виборців, внесених до витягу для голос. за місцем перебув.",4.К-сть невикор. бюлетенів,"5.К-сть виборців, які отримали бюлетені у приміщ.","6.К-сть виборців, які отримали бюлетені за місцем перебув.",...,"8.К-сть бюлетенів, що не підлягають врахуванню","9.К-сть виборців, які взяли участь у голосуванні","10.К-сть бюлетенів, визнаних недійсними",ЗеленськийВолодимир_round2,ПорошенкоПетро_round2,Дата та час протоколу,"2.К-сть виборців, внесених до списку",reg_voters_change,reg_voters_change_abs,participation_round2
0,050130,11,"Вінницька область, Територіальний виборчий окр...","Школа-ліцей, спортивна зала",1696,1694,103.0,602,1004,90,...,0,1094,16,579.0,499.0,22.04.2019 00:30,1686,8,8,0.639383
1,050131,11,"Вінницька область, Територіальний виборчий окр...","ДНЗ ""Казка"", спортивна зала",2128,2095,15.0,822,1295,11,...,0,1306,28,751.0,527.0,22.04.2019 00:20,2109,-14,14,0.605974
2,050132,11,"Вінницька область, Територіальний виборчий окр...","ПСП ""Агрофірма Батьківщина"", приіщення контори",1296,1281,4.0,484,808,4,...,0,812,14,471.0,327.0,22.04.2019 01:15,1282,-1,1,0.622465
3,050133,11,"Вінницька область, Територіальний виборчий окр...","школа, спортзал",2448,2422,14.0,933,1504,11,...,0,1515,37,769.0,709.0,22.04.2019 01:30,2434,-12,12,0.607231
4,050134,11,"Вінницька область, Територіальний виборчий окр...","звіргосподарство, кабінет головного бухгалтера",240,222,0.0,108,132,0,...,0,132,0,77.0,55.0,21.04.2019 22:10,225,-3,3,0.586667


In [3]:
df_round1['participation_round1'] = (df_round1['9.К-сть виборців, які взяли участь у голосуванні'] - 
                                     df_round1['10.К-сть бюлетенів, визнаних недійсними'])/df_round1['2.К-сть виборців, внесених до списку']
df_round1.head()

,district_number,district_gen_numeric,district_general,district_location,1.К-сть одержаних бюлетенів,"2.К-сть виборців, внесених до списку","3.К-сть виборців, внесених до витягу для голос. за місцем перебув.",4.К-сть невикор. бюлетенів,"5.К-сть виборців, які отримали бюлетені у приміщ.","6.К-сть виборців, які отримали бюлетені за місцем перебув.",...,СкоцикВіталій,СмешкоІгор,СоловйовОлександр,ТарутаСергій,ТимошенкоЮлія,ТимошенкоЮрій,ШевченкоІгор,ШевченкоОлександр,Дата та час протоколу,participation_round1
0,050130,11,"Вінницька область, Територіальний виборчий окр...","Школа-ліцей, спортивна зала",1704,1686,112,583,1031,90,...,1,120,0,0,169,11,1,3,01.04.2019 01:30,0.653025
1,050131,11,"Вінницька область, Територіальний виборчий окр...","ДНЗ ""Казка"", спортивна зала",2148,2109,17,790,1348,10,...,2,147,0,2,213,19,0,9,01.04.2019 00:50,0.635846
2,050132,11,"Вінницька область, Територіальний виборчий окр...","ПСП ""Агрофірма Батьківщина"", приіщення контори",1294,1282,4,453,837,4,...,0,95,2,0,104,9,1,1,02.04.2019 23:45,0.646646
3,050133,11,"Вінницька область, Територіальний виборчий окр...","школа, спортзал",2439,2434,15,845,1582,12,...,3,186,0,0,231,10,1,12,01.04.2019 19:00,0.649959
4,050134,11,"Вінницька область, Територіальний виборчий окр...","звіргосподарство, кабінет головного бухгалтера",222,225,0,82,140,0,...,0,14,0,0,26,0,1,0,01.04.2019 00:10,0.622222


In [4]:
base_cols = ['district_number', 'district_gen_numeric']

other_cols = ['participation_round1']

main_cands = ['БойкоЮрій',
             'ВілкулОлександр',
             'ГриценкоАнатолій',
             'ЛяшкоОлег',
             'СмешкоІгор',
             'ТимошенкоЮлія',
             'other_cands']

other_cands = ['БалашовГеннадій',
               'БезсмертнийРоман', 
               'БогомолецьОльга', 
               'БогословськаІнна',
               'БондарВіктор',
               'ВащенкоОлександр',
               'ГаберМикола',
               'ДанилюкОлександр',
               'Дерев’янкоЮрій',
               'ЖуравльовВасиль',
               'КаплінСергій',
               'КармазінЮрій',
               'КиваІлля',
               'КорнацькийАркадій',
               'КошулинськийРуслан',
               'КривенкоВіктор',
               'КупрійВіталій',
               'ЛитвиненкоЮлія',
               'МорозОлександр',
               'НаливайченкоВалентин',
               'НасіровРоман',
               'НовакАндрій',
               'НосенкоСергій',
               'ПетровВолодимир',
               'РиговановРуслан',
               'СкоцикВіталій',
               'СоловйовОлександр',
               'ТарутаСергій',
               'ТимошенкоЮрій',
               'ШевченкоІгор',
               'ШевченкоОлександр']

df_round1['other_cands'] = df_round1[other_cands].sum(axis=1)
df_round1['participation_round1'] = (df_round1['9.К-сть виборців, які взяли участь у голосуванні'] - df_round1['10.К-сть бюлетенів, визнаних недійсними'])/df_round1['2.К-сть виборців, внесених до списку']
df_candidates1 = df_round1[base_cols+other_cols+candidates+main_cands].copy()
df_candidates1.loc[:,'pool_round1'] = df_candidates1.loc[:,main_cands].sum(axis=1)
print(df_candidates1.shape)
df_candidates1.head()

(29989, 13)


,district_number,district_gen_numeric,participation_round1,ЗеленськийВолодимир,ПорошенкоПетро,БойкоЮрій,ВілкулОлександр,ГриценкоАнатолій,ЛяшкоОлег,СмешкоІгор,ТимошенкоЮлія,other_cands,pool_round1
0,050130,11,0.653025,232,370,32,11,73,45,120,169,49,499
1,050131,11,0.635846,319,365,47,33,97,46,147,213,74,657
2,050132,11,0.646646,204,247,41,10,48,35,95,104,38,371
3,050133,11,0.649959,320,524,55,29,94,61,186,231,83,739
4,050134,11,0.622222,36,39,1,1,12,3,14,26,8,65


In [5]:
df_all = df_results.merge(df_candidates1,on=base_cols)
print(df_all.shape)
print(df_all.columns)
df_all.head()

(29608, 32)
Index(['district_number', 'district_gen_numeric', 'district_general',
       'district_location', '1.К-сть одержаних бюлетенів',
       '2.К-сть виборців, внесених до списку_x',
       '3.К-сть виборців, внесених до витягу для голос. за місцем перебув.',
       '4.К-сть невикор. бюлетенів',
       '5.К-сть виборців, які отримали бюлетені у приміщ.',
       '6.К-сть виборців, які отримали бюлетені за місцем перебув.',
       '7.Загальна к-сть виборців, які отримали бюлетені',
       '8.К-сть бюлетенів, що не підлягають врахуванню',
       '9.К-сть виборців, які взяли участь у голосуванні',
       '10.К-сть бюлетенів, визнаних недійсними', 'ЗеленськийВолодимир_round2',
       'ПорошенкоПетро_round2', 'Дата та час протоколу',
       '2.К-сть виборців, внесених до списку', 'reg_voters_change',
       'reg_voters_change_abs', 'participation_round2', 'participation_round1',
       'ЗеленськийВолодимир', 'ПорошенкоПетро', 'БойкоЮрій', 'ВілкулОлександр',
       'ГриценкоАнатолій', 

,district_number,district_gen_numeric,district_general,district_location,1.К-сть одержаних бюлетенів,"2.К-сть виборців, внесених до списку_x","3.К-сть виборців, внесених до витягу для голос. за місцем перебув.",4.К-сть невикор. бюлетенів,"5.К-сть виборців, які отримали бюлетені у приміщ.","6.К-сть виборців, які отримали бюлетені за місцем перебув.",...,ЗеленськийВолодимир,ПорошенкоПетро,БойкоЮрій,ВілкулОлександр,ГриценкоАнатолій,ЛяшкоОлег,СмешкоІгор,ТимошенкоЮлія,other_cands,pool_round1
0,050130,11,"Вінницька область, Територіальний виборчий окр...","Школа-ліцей, спортивна зала",1696,1694,103.0,602,1004,90,...,232,370,32,11,73,45,120,169,49,499
1,050131,11,"Вінницька область, Територіальний виборчий окр...","ДНЗ ""Казка"", спортивна зала",2128,2095,15.0,822,1295,11,...,319,365,47,33,97,46,147,213,74,657
2,050132,11,"Вінницька область, Територіальний виборчий окр...","ПСП ""Агрофірма Батьківщина"", приіщення контори",1296,1281,4.0,484,808,4,...,204,247,41,10,48,35,95,104,38,371
3,050133,11,"Вінницька область, Територіальний виборчий окр...","школа, спортзал",2448,2422,14.0,933,1504,11,...,320,524,55,29,94,61,186,231,83,739
4,050134,11,"Вінницька область, Територіальний виборчий окр...","звіргосподарство, кабінет головного бухгалтера",240,222,0.0,108,132,0,...,36,39,1,1,12,3,14,26,8,65


In [6]:
#stage 1: find out, who stayed home or mobilized
df_all['stayed_home'] = df_all['participation_round1']*df_all['2.К-сть виборців, внесених до списку']-df_all['participation_round2']*df_all['2.К-сть виборців, внесених до списку']
df_all['mobilized'] = df_all['stayed_home'].map(lambda x: x*(-1) if x<0 else 0)
df_all['stayed_home'] = df_all['stayed_home'].map(lambda x: x if x>0 else 0)

df_all['voted_round1'] = df_all['2.К-сть виборців, внесених до списку']*df_all['participation_round1']
df_all['voted_round2'] = df_all['2.К-сть виборців, внесених до списку']*df_all['participation_round2']

for candidate in candidates:
    df_all[candidate+'_stayed_home'] = df_all.apply(lambda x: x[candidate]-x[candidate+'_round2'] if ((x[candidate]-x[candidate+'_round2'])>0)&(x['stayed_home']>0) else 0,axis=1)

df_all['stayed_home_others'] = df_all['stayed_home']-df_all[candidates[0]+'_stayed_home']-df_all[candidates[1]+'_stayed_home']
    
for candidate in main_cands:
    share = df_all[candidate]/df_all['pool_round1']
    df_all[candidate+'_stayed_home'] = share*df_all['stayed_home_others']

def mobilization(x):
    share_zel = x[candidates[0]+'_round2']/x['voted_round2']
    if (x[candidates[0]]>x[candidates[0]+'_round2']):
        return (0,x['mobilized'])
    elif (x[candidates[1]]>x[candidates[1]+'_round2']):
        return(x['mobilized'],0)
    elif (x[candidates[0]]+x['mobilized']*share_zel)>x[candidates[0]+'_round2']:
        part_mobilization = x[candidates[0]+'_round2']-x[candidates[0]]
        return(part_mobilization,x['mobilized']-part_mobilization)
    elif (x[candidates[1]]+x['mobilized']*(1-share_zel))>+x[candidates[1]+'_round2']:
        part_mobilization = x[candidates[1]+'_round2']-x[candidates[1]]
        return(x['mobilized']-part_mobilization,part_mobilization)
    else:
        return(x['mobilized']*share_zel,x['mobilized']*(1-share_zel))

df_all[candidates[0]+'_mobilized'],df_all[candidates[1]+'_mobilized'] = zip(*df_all.apply(lambda x: mobilization(x), axis=1))
    
print(f'Shape after stage 1: {df_all.shape}')

#stage 2: find out, where votes outflowed to the competitor
for candidate in candidates:
    share = df_all[candidate]/df_all['voted_round1']
    df_all[candidate+'_outflow'] = df_all[candidate]-df_all[candidate+'_round2']-df_all[candidate+'_stayed_home']
    df_all[candidate+'_outflow'] = df_all[candidate+'_outflow'].map(lambda x: x if x>0 else 0)
    
df_all['ЗеленськийВолодимир_inflow'] = df_all['ПорошенкоПетро_outflow']
df_all['ПорошенкоПетро_inflow'] = df_all['ЗеленськийВолодимир_outflow']
print(f'Shape after stage 2: {df_all.shape}')

#stage 3: find out, how the votes from the other candidates inflowed to the candidate
for candidate in candidates:
    df_all[candidate+'_inflow_others'] = df_all[candidate+'_round2']-df_all[candidate]-df_all[candidate+'_inflow']-df_all[candidate+'_mobilized']+df_all[candidate+'_outflow']+df_all[candidate+'_stayed_home']
    for candidate_lost in main_cands:
        share = df_all[candidate_lost]/df_all['pool_round1']
        df_all[candidate+'_inflow_'+candidate_lost] = share*df_all[candidate+'_inflow_others']
print(f'Shape after stage 3: {df_all.shape}')

df_all.head()

Shape after stage 1: (29608, 48)
Shape after stage 2: (29608, 52)
Shape after stage 3: (29608, 68)


,district_number,district_gen_numeric,district_general,district_location,1.К-сть одержаних бюлетенів,"2.К-сть виборців, внесених до списку_x","3.К-сть виборців, внесених до витягу для голос. за місцем перебув.",4.К-сть невикор. бюлетенів,"5.К-сть виборців, які отримали бюлетені у приміщ.","6.К-сть виборців, які отримали бюлетені за місцем перебув.",...,ЗеленськийВолодимир_inflow_ТимошенкоЮлія,ЗеленськийВолодимир_inflow_other_cands,ПорошенкоПетро_inflow_others,ПорошенкоПетро_inflow_БойкоЮрій,ПорошенкоПетро_inflow_ВілкулОлександр,ПорошенкоПетро_inflow_ГриценкоАнатолій,ПорошенкоПетро_inflow_ЛяшкоОлег,ПорошенкоПетро_inflow_СмешкоІгор,ПорошенкоПетро_inflow_ТимошенкоЮлія,ПорошенкоПетро_inflow_other_cands
0,050130,11,"Вінницька область, Територіальний виборчий окр...","Школа-ліцей, спортивна зала",1696,1694,103.0,602,1004,90,...,117.521042,34.074148,129.0,8.272545,2.843687,18.871743,11.633267,31.022044,43.689379,12.667335
1,050131,11,"Вінницька область, Територіальний виборчий окр...","ДНЗ ""Казка"", спортивна зала",2128,2095,15.0,822,1295,11,...,140.054795,48.657534,162.0,11.589041,8.136986,23.917808,11.342466,36.246575,52.520548,18.246575
2,050132,11,"Вінницька область, Територіальний виборчий окр...","ПСП ""Агрофірма Батьківщина"", приіщення контори",1296,1281,4.0,484,808,4,...,74.846361,27.347709,80.0,8.840970,2.156334,10.350404,7.547170,20.485175,22.425876,8.194070
3,050133,11,"Вінницька область, Територіальний виборчий окр...","школа, спортзал",2448,2422,14.0,933,1504,11,...,140.350474,50.428958,185.0,13.768606,7.259811,23.531800,15.270636,46.562923,57.828146,20.778078
4,050134,11,"Вінницька область, Територіальний виборчий окр...","звіргосподарство, кабінет головного бухгалтера",240,222,0.0,108,132,0,...,16.400000,5.046154,16.0,0.246154,0.246154,2.953846,0.738462,3.446154,6.400000,1.969231


In [7]:
'''
Basic checks
'''
#voters quantity check
print(f"Voters check round 1 error: {df_all['voted_round1'].sum() - sum([df_all[i].sum() for i in ['ЗеленськийВолодимир', 'ПорошенкоПетро','pool_round1']])}")
print(f"Voters check round 2 error: {df_all['voted_round2'].sum() - sum([df_all[candidate+'_round2'].sum() for candidate in candidates])}")

#mobilization check
print(f"Mobilization between rounds error: {df_all['voted_round2'].sum() - (df_all['voted_round1'].sum() + df_all['mobilized'].sum() - df_all['stayed_home'].sum())}")
if df_all['mobilized'].sum() == df_all[candidates[0]+'_mobilized'].sum() + df_all[candidates[1]+'_mobilized'].sum():
    print(f'Mobilization distribution check is successful')
else:
    print(f"Mobilization distribution check error: {df_all['mobilized'].sum() - (df_all[candidates[0]+'_mobilized'].sum() + df_all[candidates[1]+'_mobilized'].sum())}")

for candidate in candidates:
    #negative values check
    if len(df_all[candidate].loc[df_all[candidate]<-1])>0:
        print(f'Round1 data on {candidate} has negtative values')
        break
    elif len(df_all[candidate+'_inflow'].loc[df_all[candidate+'_inflow']<-1])>0:
        print(f'Inflow data on {candidate} has negtative values')
        break
    elif len(df_all[candidate+'_inflow_others'].loc[df_all[candidate+'_inflow_others']<-1])>0:
        print(f'Inflow (others) data on {candidate} has negtative values')
        break
    elif len(df_all[candidate+'_stayed_home'].loc[df_all[candidate+'_stayed_home']<-1])>0:
        print(f'Stayed_home data on {candidate} has negtative values')
        break
    elif len(df_all[candidate+'_outflow'].loc[df_all[candidate+'_outflow']<-1])>0:
        print(f'Outflow data on {candidate} has negtative values')
        break

    print(f'Negative values check for {candidate} is successful')
    
    #decomposition check
    round1 = df_all[candidate].sum()
    round2 = df_all[candidate+'_round2'].sum()

    inflow = df_all[candidate+'_inflow'].sum()
    inflow_others = df_all[candidate+'_inflow_others'].sum()
    mobilized = df_all[candidate+'_mobilized'].sum()

    home = df_all[candidate+'_stayed_home'].sum()
    outflow = df_all[candidate+'_outflow'].sum()

    if round2 == round1 + inflow + inflow_others + mobilized - home - outflow:
        print(f'Voters decomposition for candidate {candidate} is successful')
    else:
        print(f'Voters decomposition for candidate {candidate}: {round2-(round1 + inflow + inflow_others + mobilized - home - outflow)} error')

    #inflows check
    check_inflows = 0
    for candidate_lost in main_cands:
        check_inflows += df_all[candidate+'_inflow_'+candidate_lost].sum()

    if inflow_others == check_inflows:
        print(f'Inflows test for candidate {candidate} is successful')
    else:
        print(f'Inflows test for candidate {candidate}: {inflow_others-check_inflows} error')

Voters check round 1 error: 99.0
Voters check round 2 error: 0.0
Mobilization between rounds error: 0.0
Mobilization distribution check is successful
Negative values check for ЗеленськийВолодимир is successful
Voters decomposition for candidate ЗеленськийВолодимир is successful
Inflows test for candidate ЗеленськийВолодимир: -9.313225746154785e-10 error
Negative values check for ПорошенкоПетро is successful
Voters decomposition for candidate ПорошенкоПетро is successful
Inflows test for candidate ПорошенкоПетро: 2.3283064365386963e-10 error


In [59]:
df_all.to_csv('00_data/elections_round2_all_v2.csv',index=False)

In [36]:
for candidate in main_cands:
    df_all_copy[candidate+'_inflow'] = df_all_copy[candidate] - df_all_copy[candidate+'_stayed_home']

string = ' + '
cands_inflow = [cand+'_inflow' for cand in main_cands]

results = smf.ols('ЗеленськийВолодимир_inflow_others ~ ' + string.join(cands_inflow), data=df_all_copy).fit()
print(results.summary())

                                    OLS Regression Results                                   
Dep. Variable:     ЗеленськийВолодимир_inflow_others   R-squared:                       0.985
Model:                                           OLS   Adj. R-squared:                  0.985
Method:                                Least Squares   F-statistic:                 2.825e+05
Date:                               Fri, 26 Apr 2019   Prob (F-statistic):               0.00
Time:                                       17:01:05   Log-Likelihood:            -1.3640e+05
No. Observations:                              29606   AIC:                         2.728e+05
Df Residuals:                                  29598   BIC:                         2.729e+05
Df Model:                                          7                                         
Covariance Type:                           nonrobust                                         
                              coef    std err          t    

In [26]:
results2 = smf.ols('ПорошенкоПетро_inflow_others ~ ' + string.join(cands_inflow), data=df_all_copy).fit()
print(results2.summary())

                                 OLS Regression Results                                 
Dep. Variable:     ПорошенкоПетро_inflow_others   R-squared:                       0.868
Model:                                      OLS   Adj. R-squared:                  0.868
Method:                           Least Squares   F-statistic:                 2.779e+04
Date:                          Fri, 26 Apr 2019   Prob (F-statistic):               0.00
Time:                                  16:03:44   Log-Likelihood:            -1.3640e+05
No. Observations:                         29606   AIC:                         2.728e+05
Df Residuals:                             29598   BIC:                         2.729e+05
Df Model:                                     7                                         
Covariance Type:                      nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
------------------

In [37]:
bounds = [(0,1) for cand in main_cands]
bounds

[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]

In [73]:
X = df_all_copy[cands_inflow]
print(X.shape)

# Define the Model
model = lambda b, X: b[0] * X.iloc[:,0] + b[1] * X.iloc[:,1] + b[2] * X.iloc[:,2] + b[3] * X.iloc[:,3] + b[4] * X.iloc[:,4] + b[5] * X.iloc[:,5] + b[6] * X.iloc[:,6]

# The objective Function to minimize (least-squares regression)
obj = lambda b, Y, X: np.sum(np.abs(Y-model(b, X))**2)

# Bounds: b[0], b[1], b[2] >= 0
bnds = [(0, 1) for i in cands_inflow]

# Initial guess for b[1], b[2], b[3]:
xinit = np.zeros(len(cands_inflow))

for candidate in candidates:

    Y = df_all_copy[candidate+'_inflow_others']
    res = minimize(obj, args=(Y, X), x0=xinit, bounds=bnds)
    
    print(candidate + ": " + string.join([main_cands[i] + " " + str(res.x[i]) for i in range(len(main_cands))]))

(29608, 7)
ЗеленськийВолодимир: БойкоЮрій 1.0 + ВілкулОлександр 1.0 + ГриценкоАнатолій 0.0 + ЛяшкоОлег 1.0 + СмешкоІгор 1.0 + ТимошенкоЮлія 1.0 + other_cands 0.586586602633736
ПорошенкоПетро: БойкоЮрій 0.0 + ВілкулОлександр 0.0 + ГриценкоАнатолій 1.0 + ЛяшкоОлег 0.0 + СмешкоІгор 0.0 + ТимошенкоЮлія 0.0 + other_cands 0.41351633725327563


In [64]:
X.iloc[:,2]

0         73
1         97
2         48
3         94
4         12
5        109
6         88
7         19
8         28
9         15
10        28
11        11
12        11
13        21
14        87
15        69
16        18
17        25
18       100
19        42
20         0
21       131
22        24
23         4
24         2
25         5
26       138
27       126
28        93
29        84
        ... 
29578     43
29579    148
29580     38
29581    141
29582      6
29583      7
29584     14
29585      9
29586      1
29587     34
29588     14
29589      3
29590     33
29591    165
29592      6
29593    423
29594     92
29595      4
29596     44
29597     29
29598     25
29599     30
29600      1
29601     48
29602     10
29603     78
29604     95
29605      6
29606      9
29607     34
Name: ГриценкоАнатолій, Length: 29608, dtype: int64